# Procedures script to read the raw building list from a Telecommunication service provider and process the data

In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

1- Reading the buildings list

In [3]:
list = pd.read_excel('Lit Building Inventory - Public.xlsx', skiprows=3)

2- Reformat the buildings list and prepare it for SmartyStreets

In [4]:
reformat_list = pd.DataFrame({'Street Address':list['Lit Building List'], 'City':list['City'], 'State':list['State'], 'Zip Code':list['Postal']})
reformat_list.to_csv(f'SmartyStreets\input.csv')

3- Run SmartyStreets from its folder

4- Reading the result list of SmartyStreets

In [2]:
sm_list = pd.read_csv(f'SmartyStreets\output.csv')
sm_list = sm_list.fillna('').astype(str)

5- Preparing the new address formula to be ready for geocoding

In [4]:
sm_list['new address'] = sm_list.apply(lambda i: i['[primary_number]']+' '+i['[street_predirection]']+''+i['[street_name]']+' '+i['[street_suffix]']+' '+i['[street_postdirection]'], axis=1)
sm_list['new city'] = sm_list['[default_city_name]']
sm_list['new state'] = sm_list['[state_abbreviation]']
sm_list['new zipcode'] = sm_list['[zipcode]']

6- Checking for bad addresses

In [10]:
for idx, row in sm_list.iterrows():
    # Check if the address misssing the primary number
    if row['new address'][0].isdigit() == False:
        print(row['new address'])
    # Check if street name is missing
    if len(row['[street_name]']) == 0:
        print(row['[street_name]'])
    # Check if street name contains a 'PO box'
    if 'PO box' in (row['[street_name]']):
        print(idx)

7- Filtering dublicates

In [8]:
dups = sm_list.loc[sm_list.duplicated(subset=['new address', 'new state', 'new zipcode'])]
dups.to_excel('duplicated.xlsx', columns=['new address', 'new state', 'new zipcode'])
sm_list = sm_list.drop_duplicates(subset=['new address', 'new state', 'new zipcode'], keep='first')
sm_list['ID'] = sm_list.index+1

8- Prepare the input list of geocoding

In [9]:
gc_input = pd.DataFrame({'ID':sm_list.index+1,
                         'Address':sm_list['new address'],
                         'City':sm_list['new city'],
                         'State':sm_list['new state'],
                         'Country':'USA',
                         'Postal':sm_list['new zipcode']})

gc_input.to_csv(f'c2f_geocoder\c2f_geocoder\gc_input.csv', index=False)

9- Run the geocoding script

10- Reading the result list of geocoding

In [10]:
gc_output = pd.read_csv(f'c2f_geocoder\c2f_geocoder\gc_output.csv')
gc_output.rename(columns={'id':'ID'}, inplace=True)

11- Filtering the good addresses

In [12]:
good_addresses = gc_output.loc[(gc_output['accuracy_type'] == 'house number') & (gc_output['accuracy'] >= 0.8)]
good_addresses[['lat','long','Postal']].astype(str)
good_addresses = good_addresses.drop('ID', axis=1)

12- Check for bad addresses that we could take their Lat/Long values from SamrtStreets result list

In [ ]:
bad_adds_list = []
for idx, row in gc_output.iterrows():
    if (row['accuracy_type'] != 'house number') & (row['accuracy'] >= 0.8):
        sm_adds = sm_list[sm_list['new address'] == row['address']]
        sm_adds['accuracy'] = row['accuracy']
        sm_adds['accuracy_type'] = row['accuracy_type']
        sm_adds['country'] = 'USA'
        sm_adds = sm_adds[['new address', 'new city', 'new state', 'country', '[latitude]', '[longitude]', 'accuracy', 'accuracy_type', 'new zipcode']]
        columns_map = {'new address':'address', 'new city':'city', 'new state':'state', '[latitude]':'lat', '[longitude]':'long', 'new zipcode':'Postal'}
        sm_adds.rename(columns=columns_map, inplace=True)
        new_good = pd.concat([good_addresses, sm_adds], axis=0)
        new_good.to_csv(f'c2f_geocoder\c2f_geocoder\good.csv')
    # Check if there is bad addresses which its [accuracy] < 0.8 even if its [accuracy_type] == 'house number'
    if (row['accuracy_type'] == 'house number') & (row['accuracy'] < 0.8):
        bad_adds_list.append(row)
        bad_adds = pd.DataFrame(bad_adds_list, columns=gc_output.columns)
        bad_adds.to_csv(f'c2f_geocoder\c2f_geocoder\Bad.csv')
        print('there are bad addresses, and it saved in a csv file')

13- Export good addresses in a new csv file

In [14]:
good_addresses.to_csv(f'c2f_geocoder\c2f_geocoder\good.csv')

In [15]:
clli = gpd.read_file(f'clli_boundary\clli_boundary.shp')[['sw_clli', 'npa', 'nxx', 'lata', 'geometry']]

In [16]:
blg_list_geom = [Point(xy) for xy in zip(good_addresses['long'], good_addresses['lat'])]
blg_list_gdf = gpd.GeoDataFrame(good_addresses, crs='EPSG:4326', geometry=blg_list_geom)

14- Implement a spatial join to add the needed data from columns ('sw_clli', 'npa', 'nxx', 'lata')

In [19]:
join = gpd.sjoin(blg_list_gdf, clli, how='left', predicate='within')

In [20]:
result = join.drop_duplicates(subset='address', keep='first')